<a href="https://colab.research.google.com/github/JSJeong-me/JBNU-2021/blob/main/Predictive_Analytics/decision_tree/3_NeuralNetwork9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense

In [2]:
# import XLS file
path1 = "./credit_cards_dataset.csv"
path2 = "./Data_USD.csv"

data1 = pd.read_csv(path1) # skip the first row of the spreadsheet
data2 = pd.read_csv(path2) # skip the first row of the spreadsheet
# remove spaces from target column name

In [3]:
data2['ID']=np.nan

In [4]:
for i in range(0,30000):
    data2.loc[i,['ID']]=i+1

In [5]:
data2=data2.drop('DEFAULT_PAYMENT_NEXT_MO', axis =1)

In [6]:
#pd.merge(data1, data2, on="key", how="outer")
data_sum = pd.merge(data2, data1, on="ID", how="left")

In [7]:
data_sum = data_sum.drop('ID', axis =1)

In [8]:
df=data_sum

In [9]:
# X = df.drop('Y_Value',axis =1).values
# y = df['Y_Value'].values
X = df.drop('default.payment.next.month',axis =1).values
X[2999,0]

1814.285714

In [10]:
X.shape

(30000, 33)

In [11]:
y = df['default.payment.next.month'].values
#y.reshape(-1,1)

In [ ]:
#print(X.shape)
X.shape

(30000, 10)

In [ ]:
#print(y.shape)
y.shape

(30000,)

In [12]:
X_train, X_test, y_train, y_test = train_test_split (X,y,test_size=0.3, random_state=42)

In [14]:
y_test.T

array([0, 0, 0, ..., 0, 0, 0])

In [13]:
X_test.shape

(9000, 33)

In [15]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)

In [ ]:
X_scaler

In [16]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
X_train_scaled

In [17]:
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [18]:
from keras.models import Sequential

#instantiate
model = Sequential()

In [19]:
from keras.layers import Dense, Dropout

number_inputs = X_train.shape[1]
number_hidden = 30

model.add(Dense(units = number_hidden, activation ='relu', input_dim=number_inputs))
model.add(Dense(units = 35, activation ='relu')) #second hidden layer
model.add(Dense(units = 25, activation ='relu')) #second hidden layer
model.add(Dropout(0.30))
model.add(Dense(units = 15, activation ='relu')) #second hidden layer
model.add(Dropout(0.30))
model.add(Dense(units = 5, activation ='relu')) #third hidden layer

In [20]:
number_classes =2 ## yes or no
model.add(Dense(units = number_classes, activation = 'sigmoid'))

In [ ]:
model.summary()

In [23]:
#compile the model
model.compile(optimizer = 'sgd' ,
             loss = 'categorical_crossentropy',
             metrics =['accuracy'])

In [ ]:
#train the model

model.fit(X_train_scaled, y_train_categorical, epochs=200,shuffle = True,verbose =2)

In [ ]:
model.save("ccneuralnetwork.h5")

In [ ]:
#quantify the model
model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test_categorical,verbose =2)
print( model_loss )
print (model_accuracy)

F1, Precision Recall, and Confusion Matrix

In [26]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report

In [ ]:
y_prediction = model.predict_classes(X_test)

In [ ]:
y_prediction.reshape(-1,1)

In [29]:
print("Recall score:"+ str(recall_score(y_test, y_prediction)))

Recall score:0.1377551020408163


In [ ]:
print(classification_report(y_test, y_prediction,
                            target_names=["default", "non_default"]))

In [31]:
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="red" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_prediction)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Defualt', 'Non_default'],
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Defualt', 'Non_default'], normalize=True,
                      title='Normalized confusion matrix')

plt.show()